In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

In [2]:
def transform_data(X):
    """
    This function transforms the 5 input features of matrix X (x_i denoting the i-th component of X) 
    into 21 new features phi(X) in the following manner:
    5 linear features: phi_1(X) = x_1, phi_2(X) = x_2, phi_3(X) = x_3, phi_4(X) = x_4, phi_5(X) = x_5
    5 quadratic features: phi_6(X) = x_1^2, phi_7(X) = x_2^2, phi_8(X) = x_3^2, phi_9(X) = x_4^2, phi_10(X) = x_5^2
    5 exponential features: phi_11(X) = exp(x_1), phi_12(X) = exp(x_2), phi_13(X) = exp(x_3), phi_14(X) = exp(x_4), phi_15(X) = exp(x_5)
    5 cosine features: phi_16(X) = cos(x_1), phi_17(X) = cos(x_2), phi_18(X) = cos(x_3), phi_19(X) = cos(x_4), phi_20(X) = cos(x_5)
    1 constant features: phi_21(X)=1

    Parameters
    ----------
    X: matrix of floats, dim = (700,5), inputs with 5 features

    Returns
    ----------
    X_transformed: array of floats: dim = (700,21), transformed input with 21 features
    """
    X_quad = np.square(X)
    X_expo = np.exp(X)
    X_cos = np.cos(X)
    X_const = np.ones((np.shape(X)[0],1))
    X_transformed = np.hstack([X, X_quad, X_expo, X_cos, X_const])
    assert X_transformed.shape == (np.shape(X)[0], 21)
    return X_transformed

In [3]:
def fit(X, y, lam):
    """
    This function receives training data points, transform them, and then fits the linear regression on this 
    transformed data. Finally, it outputs the weights of the fitted linear regression. 

    Parameters
    ----------
    X: matrix of floats, dim = (700,5), inputs with 5 features
    y: array of floats, dim = (700,), input labels)

    Returns
    ----------
    w: array of floats: dim = (21,), optimal parameters of linear regression
    """
    w = np.zeros((21,))
    X_transformed = transform_data(X)

    reg = linear_model.Ridge(alpha=lam, fit_intercept=False)
    reg.fit(X_transformed,y)
    w = reg.coef_

    assert w.shape == (21,)
    return w

In [4]:
def calculate_RMSE(w, X, y):
    """This function takes test data points (X and y), and computes the empirical RMSE of 
    predicting y from X using a linear model with weights w. 

    Parameters
    ----------
    w: array of floats: dim = (13,), optimal parameters of ridge regression 
    X: matrix of floats, dim = (15,13), inputs with 13 features
    y: array of floats, dim = (15,), input labels

    Returns
    ----------
    RMSE: float: dim = 1, RMSE value
    """
    RMSE = 0
    y_pred = X.dot(w)
    MSE = mean_squared_error(y, y_pred)
    RMSE = np.sqrt(MSE)

    assert np.isscalar(RMSE)
    return RMSE

In [5]:
def average_LR_RMSE(X, y, lambdas, n_folds):
    """
    Main cross-validation loop, implementing 10-fold CV. In every iteration (for every train-test split), the RMSE for every lambda is calculated, 
    and then averaged over iterations.
    
    Parameters
    ---------- 
    X: matrix of floats, dim = (150, 13), inputs with 13 features
    y: array of floats, dim = (150, ), input labels
    lambdas: list of floats, len = 5, values of lambda for which ridge regression is fitted and RMSE estimated
    n_folds: int, number of folds (pieces in which we split the dataset), parameter K in KFold CV
    
    Returns
    ----------
    avg_RMSE: array of floats: dim = (5,), average RMSE value for every lambda
    """
    RMSE_mat = np.zeros((n_folds, len(lambdas)))
    kf = KFold(n_splits=n_folds)
    lambda_i = 0
    for i, (train, test) in enumerate(kf.split(X)):
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
        for ii, l in enumerate(lambdas):
            w = fit(X_train, y_train, l)
            RMSE_mat[i][ii] = calculate_RMSE(w, transform_data(X_test), y_test)

    avg_RMSE = np.mean(RMSE_mat, axis=0)
    #assert avg_RMSE.shape == (1000,)
    return avg_RMSE

In [6]:

# Data loading
data = pd.read_csv("train.csv")
y = data["y"].to_numpy()
data = data.drop(columns=["Id", "y"])
# print a few data samples
print(data.head())
X = data.to_numpy()

#K fold hyperparameter search
lambdas = [0.1, 1, 10, 100, 200]
lambdas_2 = [x*0.1 for x in range(10,1011)]
n_folds = 10
avg_RMSE = average_LR_RMSE(X, y, lambdas_2, n_folds)

     x1    x2    x3    x4    x5
0  0.02  0.05 -0.09 -0.43 -0.08
1 -0.13  0.11 -0.08 -0.29 -0.03
2  0.08  0.06 -0.07 -0.41 -0.03
3  0.02 -0.12  0.01 -0.43 -0.02
4 -0.14 -0.12 -0.08 -0.02 -0.08


In [20]:
#choosing best lam value
best_lam = lambdas_2[np.argmin(avg_RMSE)]

42.800000000000004

In [21]:
# Data loading
data = pd.read_csv("train.csv")
y = data["y"].to_numpy()
data = data.drop(columns=["Id", "y"])
# print a few data samples
print(data.head())

X = data.to_numpy()

w = fit(X, y, lam = best_lam)
# Save results in the required format
np.savetxt("./results_final_2.csv", w, fmt="%.12f")

     x1    x2    x3    x4    x5
0  0.02  0.05 -0.09 -0.43 -0.08
1 -0.13  0.11 -0.08 -0.29 -0.03
2  0.08  0.06 -0.07 -0.41 -0.03
3  0.02 -0.12  0.01 -0.43 -0.02
4 -0.14 -0.12 -0.08 -0.02 -0.08
